# DDP: Distributed Data Parallel

複数のGPUを活用した分散学習。

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

## 分散学習

深層学習において、必要な計算を複数のコンピュータに分散させること。いくつかの種類があって、例えばデータを分散させるものとモデルを分散させるもの、またパラメータの更新を同期するか否か、など。今回は、DDPと呼ばれる、モデルを分散させ、パラメータの更新を同期する分散学習について説明する。

\*DDPというのはPyTorchが用意したAPIの名前で、一般的な名前かと言われるとそうではないと思う。ただここではDDPと呼ぶことにする。

$N$個のデータをバッチサイズ$B$で分割し、$M=N/B$個のバッチを得たとする。

$R$個のデバイスがあるとき、DDPでは$M$個のバッチを均等に（$M/R$個ずつ）分配する。また各デバイスが同じモデルのコピーを持っているとする。学習が始まると、各デバイスで一つずつバッチを処理する。ここで、バッチを一つ処理する度に各デバイスで勾配を共有し、パラメータを更新する。パラメータが更新されたら次のバッチへ進む。これを繰り返すことで並列的な学習を行う。

勾配を共有というのは単に足し合わせているか平均をとっていると思ってよい。単純にバッチサイズが$B\times R$になったような感じ。各デバイスに同じ勾配が渡るので、更新後のパラメータもデバイス間で同じになる。

デバイス間でバッチの処理速度に違いがある場合、遅い方に合わせられる。全てのデバイスがバッチを処理するまで待つということ。

このあたりの図解が下記資料に

- https://www.cc.u-tokyo.ac.jp/events/lectures/111/20190124-1.pdf

## PyTorchでの実装

各デバイスで実行するプロセスを呼び出し可能なオブジェクトとして定義し、`torch.multiprocessing`で動かす。デバイス間での勾配の共有には`torch.nn.parallel.DistributedDataParallel`を使う。

[Getting Started with Distributed Data Parallel](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html)

### `DistributedSampler`

[torch.utils.data.distributed.DistributedSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.distributed.DistributedSampler)

データセットを分割する際に用いる。

In [2]:
from torch.utils.data import Dataset, DistributedSampler

適当なデータセットを用意。

In [17]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

ds = MyDataset(torch.arange(20))

ここで二つの`Sampler`を用意してみる。

In [18]:
sampler1 = DistributedSampler(ds, num_replicas=2, rank=0, shuffle=False)
sampler2 = DistributedSampler(ds, num_replicas=2, rank=1, shuffle=False)

これを使って二つの`DataLoader`を作成する。

In [19]:
dataloader1 = DataLoader(ds, batch_size=5, sampler=sampler1)
dataloader2 = DataLoader(ds, batch_size=5, sampler=sampler2)

for x in dataloader1:
    print(x)
print()
for x in dataloader2:
    print(x)

tensor([0, 2, 4, 6, 8])
tensor([10, 12, 14, 16, 18])

tensor([1, 3, 5, 7, 9])
tensor([11, 13, 15, 17, 19])


重複無しで綺麗に二つに分割されている。これを使ってデバイスごとにデータを分割する。

`shuffle=True`にするとランダムにデータを分割する。

In [20]:
sampler1 = DistributedSampler(ds, num_replicas=2, rank=0, shuffle=True)
sampler2 = DistributedSampler(ds, num_replicas=2, rank=1, shuffle=True)
dataloader1 = DataLoader(ds, batch_size=5, sampler=sampler1)
dataloader2 = DataLoader(ds, batch_size=5, sampler=sampler2)
for x in dataloader1:
    print(x)
print()
for x in dataloader2:
    print(x)

tensor([ 4, 13,  7,  3,  9])
tensor([11, 16, 10, 15,  1])

tensor([ 5, 19, 14,  6, 17])
tensor([ 2, 18, 12,  8,  0])


ちなみに、何回実行しても同じ分け方になる。中でシードが固定されているのだと思う。分け方を変えたい場合はepochを変える。

In [21]:
sampler1.set_epoch(1)
sampler2.set_epoch(1)
for x in dataloader1:
    print(x)
print()
for x in dataloader2:
    print(x)

tensor([ 5,  2, 19,  1,  4])
tensor([ 0, 16, 15,  6, 12])

tensor([13, 11, 18,  9,  7])
tensor([14, 10,  3, 17,  8])



データ数がデバイス数で割り切れない場合、余りの数だけ重複を発生させて数を揃える。

In [15]:
ds = MyDataset(torch.arange(21))
sampler1 = DistributedSampler(ds, num_replicas=2, rank=0, shuffle=False)
sampler2 = DistributedSampler(ds, num_replicas=2, rank=1, shuffle=False)
dataloader1 = DataLoader(ds, batch_size=5, sampler=sampler1)
dataloader2 = DataLoader(ds, batch_size=5, sampler=sampler2)
for x in dataloader1:
    print(x)
print()
for x in dataloader2:
    print(x)

tensor([0, 2, 4, 6, 8])
tensor([10, 12, 14, 16, 18])
tensor([20])

tensor([1, 3, 5, 7, 9])
tensor([11, 13, 15, 17, 19])
tensor([0])


### `DistributedDataParallel`

[Distributed Data Parallel](https://pytorch.org/docs/stable/notes/ddp.html)